Defining Core Classes

In [1]:
#Room Class
class Room:
    def __init__(self, room_number, room_type, price):
        self.room_number = room_number
        self.room_type = room_type  # E.g., single, double, suite
        self.price = price
        self.is_available = True  # Availability status

    def __str__(self):
        return f"Room {self.room_number} ({self.room_type}), Price: ${self.price}/night"


In [2]:
#Guest Class
class Guest:
    def __init__(self, name, contact_info):
        self.name = name
        self.contact_info = contact_info
        self.bookings = []  # List to store guest's bookings

    def __str__(self):
        return f"Guest: {self.name}, Contact: {self.contact_info}"


In [3]:
#Booking Class
from datetime import datetime

class Booking:
    def __init__(self, room, guest, check_in, check_out):
        self.room = room
        self.guest = guest
        self.check_in = datetime.strptime(check_in, '%Y-%m-%d')
        self.check_out = datetime.strptime(check_out, '%Y-%m-%d')
        self.is_paid = False  # Payment status

    def __str__(self):
        return (f"Booking for {self.guest.name} in Room {self.room.room_number} "
                f"from {self.check_in.date()} to {self.check_out.date()}")

    def calculate_total_cost(self):
        num_nights = (self.check_out - self.check_in).days
        return num_nights * self.room.price


CRUD Operations

In [4]:
#Room Management
class Hotel:
    def __init__(self):
        self.rooms = []

    def add_room(self, room):
        self.rooms.append(room)

    def update_room(self, room_number, room_type=None, price=None):
        room = self.find_room_by_number(room_number)
        if room:
            if room_type:
                room.room_type = room_type
            if price:
                room.price = price
            return room
        else:
            print("Room not found!")
    
    def delete_room(self, room_number):
        room = self.find_room_by_number(room_number)
        if room:
            self.rooms.remove(room)
        else:
            print("Room not found!")

    def find_room_by_number(self, room_number):
        for room in self.rooms:
            if room.room_number == room_number:
                return room
        return None


In [ ]:
#Guest Management
class Hotel:
    def __init__(self):
        self.rooms = []
        self.guests = []

    def add_guest(self, guest):
        self.guests.append(guest)

    def update_guest(self, guest_name, new_contact_info=None):
        guest = self.find_guest_by_name(guest_name)
        if guest:
            if new_contact_info:
                guest.contact_info = new_contact_info
            return guest
        else:
            print("Guest not found!")

    def delete_guest(self, guest_name):
        guest = self.find_guest_by_name(guest_name)
        if guest:
            self.guests.remove(guest)
        else:
            print("Guest not found!")

    def find_guest_by_name(self, guest_name):
        for guest in self.guests:
            if guest.name == guest_name:
                return guest
        return None


Booking Logic and Availability

In [5]:
#Availability Check
class Hotel:
    # Other methods...

    def is_room_available(self, room, check_in, check_out):
        for booking in room.bookings:
            if not (check_out <= booking.check_in or check_in >= booking.check_out):
                return False  # There's a date overlap
        return True


In [6]:
#Book a Room
class Hotel:
    # Other methods...

    def book_room(self, room_number, guest_name, check_in, check_out):
        room = self.find_room_by_number(room_number)
        guest = self.find_guest_by_name(guest_name)
        if room and guest:
            if self.is_room_available(room, check_in, check_out):
                booking = Booking(room, guest, check_in, check_out)
                room.bookings.append(booking)
                guest.bookings.append(booking)
                return booking
            else:
                print(f"Room {room_number} is not available for the selected dates.")
        else:
            print("Room or guest not found.")


Calendar View (Room Availability)

In [7]:
from datetime import timedelta

class Hotel:
    # Other methods...

    def show_room_availability(self, start_date, end_date):
        print(f"Availability from {start_date} to {end_date}:")
        for room in self.rooms:
            print(f"Room {room.room_number}: ", end="")
            available = True
            for booking in room.bookings:
                if not (end_date <= booking.check_in or start_date >= booking.check_out):
                    available = False
            if available:
                print("Available")
            else:
                print("Booked")


 Payment and Receipt Generation

In [8]:
class Booking:
    # Other methods...

    def make_payment(self, amount):
        total_cost = self.calculate_total_cost()
        if amount >= total_cost:
            self.is_paid = True
            return f"Payment of ${amount} received. Booking confirmed."
        else:
            return f"Insufficient amount. Total cost is ${total_cost}."


Database Storage

In [9]:
import sqlite3

class Database:
    def __init__(self, db_name):
        self.conn = sqlite3.connect(db_name)
        self.create_tables()

    def create_tables(self):
        self.conn.execute('''CREATE TABLE IF NOT EXISTS guest
                             (name TEXT, contact_info TEXT)''')
        self.conn.execute('''CREATE TABLE IF NOT EXISTS room
                             (room_number INTEGER, room_type TEXT, price REAL)''')
        self.conn.execute('''CREATE TABLE IF NOT EXISTS booking
                             (room_number INTEGER, guest_name TEXT, check_in TEXT, check_out TEXT)''')
    
    # CRUD methods for inserting, updating, and retrieving data...


Search Functionality

In [10]:
class Hotel:
    # Other methods...

    def search_guest(self, query):
        results = [guest for guest in self.guests if query.lower() in guest.name.lower()]
        return results

    def search_booking(self, query):
        results = [booking for booking in room.bookings for room in self.rooms 
                   if query.lower() in booking.guest.name.lower()]
        return results


In [11]:
import sqlite3

class Database:
    def __init__(self, db_name='hotel_booking.db'):
        self.conn = sqlite3.connect(db_name)
        self.create_tables()

    def create_tables(self):
        with self.conn:
            self.conn.execute('''CREATE TABLE IF NOT EXISTS rooms
                                 (room_number INTEGER PRIMARY KEY, 
                                  room_type TEXT, 
                                  price REAL)''')
            self.conn.execute('''CREATE TABLE IF NOT EXISTS guests
                                 (guest_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                  name TEXT, 
                                  contact_info TEXT)''')
            self.conn.execute('''CREATE TABLE IF NOT EXISTS bookings
                                 (booking_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                  room_number INTEGER, 
                                  guest_name TEXT, 
                                  check_in TEXT, 
                                  check_out TEXT,
                                  is_paid INTEGER,
                                  FOREIGN KEY(room_number) REFERENCES rooms(room_number),
                                  FOREIGN KEY(guest_name) REFERENCES guests(name))''')

    def add_room(self, room):
        with self.conn:
            self.conn.execute('INSERT INTO rooms (room_number, room_type, price) VALUES (?, ?, ?)',
                              (room.room_number, room.room_type, room.price))

    def add_guest(self, guest):
        with self.conn:
            self.conn.execute('INSERT INTO guests (name, contact_info) VALUES (?, ?)',
                              (guest.name, guest.contact_info))

    def add_booking(self, booking):
        with self.conn:
            self.conn.execute('''INSERT INTO bookings (room_number, guest_name, check_in, check_out, is_paid) 
                                 VALUES (?, ?, ?, ?, ?)''',
                              (booking.room.room_number, booking.guest.name, 
                               booking.check_in.strftime('%Y-%m-%d'), 
                               booking.check_out.strftime('%Y-%m-%d'), 
                               int(booking.is_paid)))

    def load_rooms(self):
        cursor = self.conn.execute('SELECT room_number, room_type, price FROM rooms')
        rooms = []
        for row in cursor:
            rooms.append(Room(row[0], row[1], row[2]))
        return rooms

    def load_guests(self):
        cursor = self.conn.execute('SELECT name, contact_info FROM guests')
        guests = []
        for row in cursor:
            guests.append(Guest(row[0], row[1]))
        return guests

    def load_bookings(self):
        cursor = self.conn.execute('SELECT room_number, guest_name, check_in, check_out, is_paid FROM bookings')
        bookings = []
        for row in cursor:
            room = self.find_room_by_number(row[0])
            guest = self.find_guest_by_name(row[1])
            booking = Booking(room, guest, row[2], row[3])
            booking.is_paid = bool(row[4])
            bookings.append(booking)
        return bookings


In [12]:
class Hotel:
    def __init__(self):
        self.db = Database()
        self.rooms = self.db.load_rooms()
        self.guests = self.db.load_guests()
        self.bookings = self.db.load_bookings()

    # Other methods like add_room, add_guest, etc.


In [13]:
def menu():
    hotel = Hotel()
    
    while True:
        print("\nHotel Booking System")
        print("1. Add Room")
        print("2. Add Guest")
        print("3. Make a Booking")
        print("4. Search for Guest")
        print("5. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            room_number = input("Enter room number: ")
            room_type = input("Enter room type: ")
            price = input("Enter price: ")
            hotel.add_room(Room(room_number, room_type, float(price)))
        elif choice == '2':
            name = input("Enter guest name: ")
            contact_info = input("Enter contact info: ")
            hotel.add_guest(Guest(name, contact_info))
        elif choice == '3':
            guest_name = input("Enter guest name: ")
            room_number = input("Enter room number: ")
            check_in = input("Enter check-in date (YYYY-MM-DD): ")
            check_out = input("Enter check-out date (YYYY-MM-DD): ")
            booking = hotel.book_room(room_number, guest_name, check_in, check_out)
            if booking:
                print(booking)
        elif choice == '4':
            guest_name = input("Enter guest name to search: ")
            guest = hotel.find_guest_by_name(guest_name)
            print(guest)
        elif choice == '5':
            print("Goodbye!")
            break


In [ ]:
import tkinter as tk

class HotelBookingApp:
    def __init__(self, root):
        self.hotel = Hotel()

        self.root = root
        self.root.title("Hotel Booking System")

        self.label = tk.Label(root, text="Welcome to the Hotel Booking System")
        self.label.pack()

        self.add_room_button = tk.Button(root, text="Add Room", command=self.add_room)
        self.add_room_button.pack()

    def add_room(self):
        room_number = input("Enter room number: ")
        room_type = input("Enter room type: ")
        price = input("Enter price: ")
        self.hotel.add_room(Room(room_number, room_type, float(price)))

root = tk.Tk()
app = HotelBookingApp(root)
root.mainloop()


In [ ]:
def generate_receipt(booking):
    total_cost = booking.calculate_total_cost()
    receipt = f"Receipt for {booking.guest.name}\nRoom: {booking.room.room_number}\n" \
              f"Check-in: {booking.check_in.date()}\nCheck-out: {booking.check_out.date()}\n" \
              f"Total: ${total_cost}\nPayment: {'Completed' if booking.is_paid else 'Pending'}"
    return receipt
